In [1]:
import lightgbm
import optuna
import pandas as pd

df = pd.read_csv('train.csv')
df

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031
...,...,...,...,...,...,...,...,...,...,...,...,...
749995,749995,Learning Lab,Episode 25,75.66,Education,69.36,Saturday,Morning,NaN,0.0,Negative,56.87058
749996,749996,Business Briefs,Episode 21,75.75,Business,35.21,Saturday,Night,NaN,2.0,Neutral,45.46242
749997,749997,Lifestyle Lounge,Episode 51,30.98,Lifestyle,78.58,Thursday,Morning,84.89,0.0,Negative,15.26000
749998,749998,Style Guide,Episode 47,108.98,Lifestyle,45.39,Thursday,Morning,93.27,0.0,Negative,100.72939


In [2]:
df['Publication_Time'].value_counts()

Publication_Time
Night        196849
Evening      195778
Afternoon    179460
Morning      177913
Name: count, dtype: int64

In [3]:
def derive_episode(episode):
    return int(episode.split()[-1])
def num(r):
    return len(str(r))
def process(df):
    df['Genre'] = df['Genre'].astype('category')
    df['Publication_Day'] = df['Publication_Day'].astype('category')
    df['Publication_Time'] = df['Publication_Time'].astype('category')
    df['Episode_Sentiment'] = df['Episode_Sentiment'].astype('category')
    df['Podcast_Name'] = df['Podcast_Name'].astype('category')
    df['Host_Popularity_percentage'] = df['Host_Popularity_percentage']
    df['Guest_Popularity_percentage'] = df['Guest_Popularity_percentage']
    df['num_decimals'] = df['Episode_Length_minutes'].apply(num)
    df['Episode_Num'] = df['Episode_Title'].apply(derive_episode)
    return df


In [4]:
df = process(df)

In [5]:
# X = df.drop(columns=['Episode_Title', 'Listening_Time_minutes'])
# y = df['Listening_Time_minutes']
#
# dtrain = lightgbm.Dataset(X,label=y,categorical_feature=['Podcast_Name','Genre','Publication_Day','Publication_Time','Episode_Sentiment'])
#
# params = {
#     "objective": "regression",
#     "metric": "rmse",
#     "learning_rate": 0.01,         # stable learning
#     "feature_fraction": 0.9,       # avoid overfitting by feature bagging
#     "bagging_fraction": 0.9,       # row bagging per tree
#     "bagging_freq": 1,             # actually apply the bagging
#     "random_state": 42,
#     "verbosity": -1
# }

#
# model = lightgbm.train(params,train_set=dtrain,num_boost_round=5000)

In [ ]:
X = df.drop(columns=['Episode_Title', 'Listening_Time_minutes'])
y = df['Listening_Time_minutes']
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np
def objective(trial):

    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        'learning_rate': trial.suggest_float("learning_rate", 0.005, 0.1),
        'num_leaves': trial.suggest_int("num_leaves", 16, 256),
        'min_data_in_leaf': trial.suggest_int("min_data_in_leaf", 10, 100),
        'feature_fraction': trial.suggest_float("feature_fraction", 0.6, 1.0),
        'bagging_fraction': trial.suggest_float("bagging_fraction", 0.6, 1.0),
        'bagging_freq': trial.suggest_int("bagging_freq", 1, 10),
        'lambda_l1': trial.suggest_float("lambda_l1", 0.0, 5.0),
        'lambda_l2': trial.suggest_float("lambda_l2", 0.0, 5.0)
    }

    kf = KFold(n_splits=5, random_state=42,shuffle=True)
    loss_list = []
    best_iters = []

    for train_idx, val_idx in kf.split(X):
        dtrain = lightgbm.Dataset(X.iloc[train_idx],label=y.iloc[train_idx])
        dvalid = lightgbm.Dataset(X.iloc[val_idx],label=y.iloc[val_idx])

        model = lightgbm.train(
            param,
            dtrain,
            num_boost_round=10000,
            valid_sets=[dvalid],
            valid_names=['valid'],
            callbacks=[
                lightgbm.early_stopping(stopping_rounds=500)
            ]
        )

        preds = model.predict(X.iloc[val_idx])
        loss = mean_squared_error(y.iloc[val_idx], preds)
        loss_list.append(loss)
        best_iters.append(model.best_iteration)

    trial.set_user_attr("best_iteration", int(np.mean(best_iters)))
    return np.mean(loss_list)

study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials=30, timeout=60*10)
full_train = lightgbm.Dataset(X,label=y)
model = lightgbm.train(study.best_trial.params,train_set=full_train,num_boost_round=study.best_trial.user_attrs['best_iteration'])

[I 2025-05-02 03:55:24,302] A new study created in memory with name: no-name-8b04c558-b391-4dd7-86f3-986be7d5cd9b


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[311]	valid's rmse: 12.9158
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[551]	valid's rmse: 12.976
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[488]	valid's rmse: 12.9639
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[339]	valid's rmse: 12.9576
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[462]	valid's rmse: 12.9119


[I 2025-05-02 03:56:55,588] Trial 0 finished with value: 167.57433816572754 and parameters: {'learning_rate': 0.07535984342570455, 'num_leaves': 234, 'min_data_in_leaf': 41, 'feature_fraction': 0.6601604784747851, 'bagging_fraction': 0.7006497414601343, 'bagging_freq': 3, 'lambda_l1': 4.847016200788929, 'lambda_l2': 3.756228299270301}. Best is trial 0 with value: 167.57433816572754.


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[150]	valid's rmse: 12.9653
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[169]	valid's rmse: 13.0255
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[249]	valid's rmse: 13.001
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[220]	valid's rmse: 12.9944
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[116]	valid's rmse: 12.9629


[I 2025-05-02 03:58:01,089] Trial 1 finished with value: 168.7361975543747 and parameters: {'learning_rate': 0.09748328828644448, 'num_leaves': 252, 'min_data_in_leaf': 22, 'feature_fraction': 0.7713339090765571, 'bagging_fraction': 0.6610111490885451, 'bagging_freq': 3, 'lambda_l1': 1.4643588327562396, 'lambda_l2': 2.9218982264483695}. Best is trial 0 with value: 167.57433816572754.


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[437]	valid's rmse: 12.9131
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[521]	valid's rmse: 12.9701
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[609]	valid's rmse: 12.9547
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[487]	valid's rmse: 12.9628
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[520]	valid's rmse: 12.9299


[I 2025-05-02 03:59:29,491] Trial 2 finished with value: 167.6029174478721 and parameters: {'learning_rate': 0.08107249493087879, 'num_leaves': 175, 'min_data_in_leaf': 82, 'feature_fraction': 0.6237277769303173, 'bagging_fraction': 0.7391178144357118, 'bagging_freq': 5, 'lambda_l1': 0.6054441355028584, 'lambda_l2': 0.9142388491491527}. Best is trial 0 with value: 167.57433816572754.


Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[297]	valid's rmse: 12.9493
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[481]	valid's rmse: 12.9984
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[517]	valid's rmse: 12.9847
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[451]	valid's rmse: 12.9766
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[347]	valid's rmse: 12.9372


[I 2025-05-02 04:00:56,989] Trial 3 finished with value: 168.2017968481121 and parameters: {'learning_rate': 0.08007129587546154, 'num_leaves': 206, 'min_data_in_leaf': 76, 'feature_fraction': 0.6545963584250651, 'bagging_fraction': 0.66267539478362, 'bagging_freq': 6, 'lambda_l1': 1.4157573370230052, 'lambda_l2': 1.8193567513326603}. Best is trial 0 with value: 167.57433816572754.


Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[9891]	valid's rmse: 12.8553
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[9774]	valid's rmse: 12.8997
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[10000]	valid's rmse: 12.8815
Training until validation scores don't improve for 500 rounds


In [ ]:
test_df = pd.read_csv('test.csv')
test_df = process(test_df)

In [ ]:
test_df['Listening_Time_minutes'] = model.predict(test_df.drop(columns=['Episode_Title']))

In [ ]:
submission = test_df[['id','Listening_Time_minutes']]
submission

In [ ]:
submission.to_csv('submission.csv',index=False)